<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/Pathminst_of_MedMNIST_QNN_AllInOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# SNIPPET 1: Setup =====================

!pip install -q medmnist pennylane pennylane-lightning torch torchvision scikit-learn pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 5.0 MB/s eta 0:00:00


In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
from torchvision import models
from torch.optim.swa_utils import AveragedModel, update_bn, SWALR
import pennylane as qml
import medmnist
from medmnist import INFO
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import scipy.integrate

warnings.filterwarnings("ignore")

# ==========================================
# 1. PATHMNIST CONFIGURATION (High Reg)
# ==========================================
CONFIG = {
    "dataset": "pathmnist",
    "img_size": 64,          # Standard Unified Input
    "D": 128,                # Embedding Dim (ResNet Stem Output)
    "q_wires": 6,            # 6 Qubits
    "q_layers": 3,           # 3 Layer Ansatz
    "head_hidden": 128,
    "dropout": 0.3,
    "n_classes": 9,          # 9 Tissue Types

    # Training
    "batch_size": 128,       # Large batch for stable gradients
    "epochs": 50,            # Needs time to converge
    "lr": 0.001,
    "wd": 1e-3,              # CRITICAL: High decay for texture generalization
    "label_smoothing": 0.1,  # Helps with 9 classes
    "seed": 42,

    # SWA
    "use_swa": True,
    "swa_start": 40,         # Late SWA
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(CONFIG['seed'])
print(f"🔧 Config: PathMNIST (9 Classes), High Regularization, SWA Enabled")

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


🔧 Config: PathMNIST (9 Classes), High Regularization, SWA Enabled


In [3]:
# ==========================================
# 2. DATA PIPELINE (Rotational Invariance)
# ==========================================
def get_transforms(img_size, is_train):
    mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

    if is_train:
        return T.Compose([
            T.Resize((img_size, img_size), interpolation=InterpolationMode.BILINEAR),
            # PATHOLOGY SPECIFIC AUGMENTATIONS
            T.RandomHorizontalFlip(p=0.5),
            T.RandomVerticalFlip(p=0.5),     # No orientation in tissue
            T.RandomRotation(90),            # Full rotation invariance
            T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
            T.ToTensor(),
            T.Normalize(mean, std)
        ])
    return T.Compose([
        T.Resize((img_size, img_size), interpolation=InterpolationMode.BILINEAR),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])

def get_loaders(config):
    info = INFO[config['dataset']]
    DataClass = getattr(medmnist, info['python_class'])

    train_ds = DataClass(split='train', transform=get_transforms(config['img_size'], True), download=True)
    val_ds = DataClass(split='val', transform=get_transforms(config['img_size'], False), download=True)
    test_ds = DataClass(split='test', transform=get_transforms(config['img_size'], False), download=True)

    # PathMNIST is fairly balanced, but sampler ensures stability
    targets = train_ds.labels.squeeze()
    class_counts = np.bincount(targets)
    class_weights = 1. / class_counts
    samples_weights = class_weights[targets]
    sampler = WeightedRandomSampler(samples_weights, len(samples_weights))

    train_loader = DataLoader(train_ds, batch_size=config['batch_size'], sampler=sampler, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=False, num_workers=2)
    test_loader = DataLoader(test_ds, batch_size=config['batch_size'], shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

print("📦 Data Pipeline Ready (Isotropic Augmentation)")

📦 Data Pipeline Ready (Isotropic Augmentation)


In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
import pennylane as qml

# ==========================================
# 3. UNIFIED ARCHITECTURE (Vectorized)
# ==========================================

class PatchEmbedding(nn.Module):
    """
    Spatial Extractor: 64x64 -> 128 Features
    Uses pretrained ResNet18 (Layers 1-2) as the 'Smart Stem'
    """
    def __init__(self):
        super().__init__()
        # Load Pretrained Weights
        backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        # Extract layers up to layer2
        self.stem = nn.Sequential(
            backbone.conv1, backbone.bn1, backbone.relu, backbone.maxpool,
            backbone.layer1,
            backbone.layer2
        )

    def forward(self, x):
        # x: [B, 3, 64, 64]
        features = self.stem(x) # Output: [B, 128, 8, 8]
        # Flatten spatial dims: [B, 128, 64] -> Transpose: [B, 64, 128]
        return features.flatten(2).transpose(1, 2)

class SimpleRNN(nn.Module):
    def __init__(self, input_dim=128, hidden_dim=128):
        super().__init__()
        # BiGRU captures texture context
        self.rnn = nn.GRU(input_dim, hidden_dim // 2, num_layers=2,
                          batch_first=True, bidirectional=True, dropout=0.2)
    def forward(self, x):
        out, _ = self.rnn(x)
        return out.mean(dim=1) # Global Average Pooling

class FastQuantumLayer(nn.Module):
    """
    Vectorized Quantum Layer
    Processes the entire batch in parallel (no for-loops)
    """
    def __init__(self, in_dim=128, n_qubits=6, n_layers=3):
        super().__init__()
        self.n_qubits = n_qubits
        self.proj6 = nn.Linear(in_dim, n_qubits)

        # 'default.qubit' supports broadcasting
        self.dev = qml.device('default.qubit', wires=n_qubits)

        self.q_params = nn.Parameter(torch.randn(n_layers, n_qubits, 2) * 0.1)
        self.n_layers = n_layers

        self.qnode = qml.QNode(self._circuit, self.dev, interface='torch')

    def _circuit(self, inputs, weights):
        # inputs shape: (Batch_Size, n_qubits)
        # We index column-wise to broadcast gates across the batch
        for i in range(self.n_qubits):
            qml.RY(inputs[:, i], wires=i)

        for l in range(self.n_layers):
            for i in range(self.n_qubits):
                qml.RY(weights[l, i, 0], wires=i)
            for i in range(self.n_qubits):
                qml.CNOT(wires=[i, (i + 1) % self.n_qubits])
            for i in range(self.n_qubits):
                qml.RZ(weights[l, i, 1], wires=i)

        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]

    def forward(self, x):
        # 1. Project & Clamp
        a6 = self.proj6(x)
        a6 = torch.clamp(a6, min=-np.pi/2, max=np.pi/2)

        # 2. Vectorized Execution
        # Returns tuple of tensors, we stack them
        q_out_tuple = self.qnode(a6, self.q_params)
        q_vals = torch.stack(q_out_tuple, dim=1).float().to(x.device)

        return q_vals, a6

class FastHybridQNN(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.patch_embed = PatchEmbedding()
        self.rnn = SimpleRNN(input_dim=128, hidden_dim=128)
        self.q_layer = FastQuantumLayer(in_dim=128, n_qubits=config['q_wires'], n_layers=config['q_layers'])

        self.head = nn.Sequential(
            nn.Linear(128 + config['q_wires'], config['head_hidden']),
            nn.ReLU(),
            nn.Dropout(config['dropout']),
            nn.Linear(config['head_hidden'], config['n_classes'])
        )
        self.cache = {}

    def forward(self, x):
        x_patch = self.patch_embed(x)
        x_classical = self.rnn(x_patch)
        x_quant, a6 = self.q_layer(x_classical)

        if not self.training:
            self.cache['a6'] = a6
            self.cache['features'] = x_classical

        combined = torch.cat([x_classical, x_quant], dim=1)
        return self.head(combined)

print("🧠 Vectorized Hybrid Model Built")

🧠 Unified Model Built (Pathology Config)


In [5]:
# ==========================================
# 4. TRAINING ENGINE
# ==========================================
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
    def forward(self, pred, target):
        log_prob = F.log_softmax(pred, dim=-1)
        model_prob = torch.ones_like(log_prob) * (self.smoothing / (pred.size(-1) - 1))
        model_prob.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
        return (-model_prob * log_prob).sum(dim=-1).mean()

def train_engine(model, train_loader, val_loader, config):
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['wd'])
    criterion = LabelSmoothingLoss(config['label_smoothing'])

    swa_model = AveragedModel(model)
    swa_scheduler = SWALR(optimizer, swa_lr=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'])

    best_acc = 0.0
    print("🚀 Training Started...")

    for epoch in range(config['epochs']):
        model.train()
        total_loss = 0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
            x, y = x.to(config['device']), y.squeeze().long().to(config['device'])
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # SWA Logic
        if config['use_swa'] and epoch >= config['swa_start']:
            swa_model.update_parameters(model)
            swa_scheduler.step()
        else:
            scheduler.step()

        model.eval()
        preds, targets = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x = x.to(config['device'])
                out = model(x)
                preds.append(out.argmax(1).cpu())
                targets.append(y.squeeze())

        acc = accuracy_score(torch.cat(targets), torch.cat(preds))
        if acc > best_acc: best_acc = acc
        print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f} | Val Acc: {acc:.4f}")

    print("🔄 SWA Finisher: Updating Batch Norm...")
    update_bn(train_loader, swa_model, device=config['device'])
    return swa_model

# --- EXECUTION ---
train_loader, val_loader, test_loader = get_loaders(CONFIG)
model = FastHybridQNN(CONFIG).to(CONFIG['device'])
final_model = train_engine(model, train_loader, val_loader, CONFIG)

# Final Metrics
final_model.eval()
preds, targets = [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(CONFIG['device'])
        out = final_model(x)
        preds.append(out.argmax(1).cpu())
        targets.append(y.squeeze())

print(f"\n🏆 Final Test Accuracy: {accuracy_score(torch.cat(targets), torch.cat(preds)):.4f}")

100%|██████████| 206M/206M [00:47<00:00, 4.36MB/s]


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 47.0MB/s]


🚀 Training Started...


Epoch 1 | Loss: 0.9275 | Val Acc: 0.8933


Epoch 2 | Loss: 0.7622 | Val Acc: 0.8566


Epoch 3 | Loss: 0.7372 | Val Acc: 0.8756


Epoch 4:  30%|██▉       | 209/704 [33:27<1:21:32,  9.88s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x78b830489ee0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1618, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/selectors.py", line 415, in 

KeyboardInterrupt: 

In [ ]:
# ==========================================
# 5. XAI ENGINE (Tri-Layer)
# ==========================================
class XAIEngine:
    def __init__(self, model, device):
        self.device = device
        self.model = model.module if hasattr(model, 'module') else model
        self.gradients, self.activations = None, None

        # Hook last conv of stem (ResNet Layer 2)
        # model.patch_embed.stem is Sequential. Index 5 is Layer2.
        # Layer2 has blocks. We hook the last block's conv2.
        self.target_layer = self.model.patch_embed.stem[5][-1].conv2
        self.target_layer.register_forward_hook(self.save_act)
        self.target_layer.register_full_backward_hook(self.save_grad)

    def save_act(self, module, input, output): self.activations = output
    def save_grad(self, module, grad_in, grad_out): self.gradients = grad_out[0]

    def normalize(self, x):
        return (x - x.min()) / (x.max() - x.min() + 1e-8)

    def grad_cam(self, x, class_idx):
        self.model.eval(); self.model.zero_grad()
        out = self.model(x)
        out[0, class_idx].backward(retain_graph=True)
        grad = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (grad * self.activations).sum(dim=1, keepdim=True)
        return self.normalize(F.interpolate(F.relu(cam), size=x.shape[2:], mode='bilinear'))

    def integrated_gradients(self, x, class_idx, steps=20):
        baseline = torch.zeros_like(x)
        x.requires_grad_(True)
        gradients = []
        for alpha in np.linspace(0, 1, steps):
            x_scaled = baseline + alpha * (x - baseline)
            x_step = x_scaled.detach().clone().requires_grad_(True)
            out = self.model(x_step)
            score = out[0, class_idx]
            self.model.zero_grad()
            score.backward()
            gradients.append(x_step.grad.detach())
        avg_grads = torch.stack(gradients).mean(dim=0)
        ig = (x - baseline) * avg_grads
        return self.normalize(ig.sum(dim=1, keepdim=True).abs())

    def q_saliency(self, x, class_idx):
        self.model.eval(); x.requires_grad_(True)
        logits = self.model(x)
        if 'a6' not in self.model.cache: _ = self.model(x)
        a6 = self.model.cache['a6']; a6.retain_grad()
        logits[0, class_idx].backward(retain_graph=True)
        da6 = a6.grad.detach().abs()
        W = self.model.q_layer.proj6.weight.detach().abs()
        S_feat = torch.matmul(da6, W)
        patch_acts = self.activations.detach().abs()
        if S_feat.shape[1] != patch_acts.shape[1]:
             S_feat = F.interpolate(S_feat.unsqueeze(0), size=patch_acts.shape[1], mode='linear').squeeze(0)
        S_patch = torch.einsum('bd,bdhw->bhw', S_feat, patch_acts).unsqueeze(1)
        return self.normalize(F.interpolate(S_patch, size=x.shape[2:], mode='bilinear'))

    def mc_dropout_entropy(self, x, T=10):
        self.model.train()
        outputs = torch.stack([torch.softmax(self.model(x), dim=1) for _ in range(T)])
        mean_p = outputs.mean(dim=0)
        return -torch.sum(mean_p * torch.log(mean_p + 1e-8), dim=1).item()

    def deletion_auc(self, x, target, saliency_map):
        B, C, H, W = x.shape
        flat_sal = saliency_map.view(B, -1)
        sorted_idx = torch.argsort(flat_sal, descending=True, dim=1)
        probs = []
        steps = np.linspace(0, 0.5, 10)
        for frac in steps:
            k = int(frac * H * W)
            mask = torch.ones_like(flat_sal)
            if k > 0: mask.scatter_(1, sorted_idx[:, :k], 0.0)
            with torch.no_grad():
                out = self.model(x * mask.view(B, 1, H, W))
                probs.append(F.softmax(out, dim=1)[0, target].item())
        return scipy.integrate.trapezoid(probs, dx=1.0/len(steps))

# Execute XAI
print("\n🔍 Generating XAI Panel...")
xai = XAIEngine(final_model, CONFIG['device'])
x, y = next(iter(test_loader))
x = x[:1].to(CONFIG['device']); target = y[0].item()

q_map = xai.q_saliency(x, target)
titles = ["Input", "Grad-CAM++", "Integrated Grads", "Q-Saliency"]
maps = [x, xai.grad_cam(x, target).detach(), xai.integrated_gradients(x, target).detach(), q_map.detach()]

plt.figure(figsize=(12, 3))
for i in range(4):
    plt.subplot(1, 4, i+1)
    img_disp = x.squeeze().detach().cpu().permute(1, 2, 0) * 0.5 + 0.5
    if i == 0: plt.imshow(img_disp)
    else:
        plt.imshow(img_disp)
        plt.imshow(maps[i].squeeze().cpu(), cmap='jet', alpha=0.5)
    plt.title(titles[i]); plt.axis('off')

print(f"   Entropy: {xai.mc_dropout_entropy(x):.4f}")
print(f"   Deletion AUC: {xai.deletion_auc(x, target, q_map):.4f}")
plt.show()